In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import pandas as pd
from sklearn.ensemble import IsolationForest

class AnomalyDetectionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Anomaly Detection and Root Cause Analysis")

        self.nominal_data = None
        self.non_nominal_data_frames = None
        self.selected_parameters = []

        self.initialize_data()
        self.create_widgets()

    def initialize_data(self):
        try:
            nominal_excel_path = "D:/Data/Book12.xlsx"
            non_nominal_excel_paths = ["D:/Data/Book13.xlsx", "D:/Data/Book14.xlsx"]
            self.nominal_data = pd.read_excel(nominal_excel_path, index_col=0)  # Use the first column as index
            self.non_nominal_data_frames = [pd.read_excel(file, index_col=0) for file in non_nominal_excel_paths]
            if self.nominal_data is not None:
                self.selected_parameters = self.nominal_data.columns.tolist()
        except Exception as e:
            messagebox.showerror("Error", f"Error loading data: {e}")

    def create_widgets(self):
        # Create frame for parameter selection
        parameter_frame = ttk.Frame(self)
        parameter_frame.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        label_parameter = ttk.Label(parameter_frame, text="Select Parameters to Plot (separated by comma):")
        label_parameter.pack(side=tk.TOP, padx=10, pady=5)

        parameter_entry = ttk.Entry(parameter_frame, width=50)
        parameter_entry.pack(side=tk.TOP, padx=10, pady=5)

        # Create plot button for parameter plot
        plot_parameter_button = ttk.Button(parameter_frame, text="Plot Selected Parameters", command=lambda: self.plot_selected_parameters(parameter_entry.get()))
        plot_parameter_button.pack(side=tk.TOP, padx=10, pady=5)

        # Create plot frame
        plot_frame = ttk.Frame(self)
        plot_frame.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        self.plot_frame = plot_frame
        self.plot_frame.clear = lambda: self.plot_frame.winfo_children()  # Clear plot frame

    def plot_selected_parameters(self, selected_parameters_str):
        selected_parameters = [param.strip() for param in selected_parameters_str.split(',')]
        for parameter in selected_parameters:
            self.plot_selected_parameter(parameter)

    def plot_selected_parameter(self, selected_parameter):
        # Check if a parameter is selected
        if not selected_parameter:
            messagebox.showerror("Error", "Please enter parameters to plot.")
            return

        # Clear previous plots
        self.plot_frame.clear()

        # Perform anomaly detection
        anomalies = self.detect_anomalies(self.nominal_data, self.non_nominal_data_frames)

        fig = Figure(figsize=(14, 10))  # Increase the figure size
        ax = fig.add_subplot(111)

        # Plot data for selected parameter across all files
        for i, non_nominal_data_frame in enumerate(self.non_nominal_data_frames):
            # Plot selected parameter
            ax.plot(non_nominal_data_frame.index, non_nominal_data_frame[selected_parameter], marker='o', linestyle='-', label=f"File {i+1}")

            # Mark anomaly points on the plot
            anomaly_indices = anomalies[i] == -1
            anomaly_points = non_nominal_data_frame[anomaly_indices]
            ax.scatter(anomaly_points.index, anomaly_points[selected_parameter], color='red', label=f'Anomaly in File {i+1}', zorder=5)

        ax.set_title(f'Parameter: {selected_parameter}')
        ax.set_xlabel('Time')  # Set x-axis label to 'Time'
        ax.set_ylabel('Value')
        ax.legend()

        # Embed the plot in the Tkinter window
        canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Add navigation toolbar for scrolling, panning, zooming, and resetting
        toolbar = NavigationToolbar2Tk(canvas, self.plot_frame)
        toolbar.update()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    def detect_anomalies(self, nominal_data, non_nominal_data_frames):
        anomalies = []
        for non_nominal_data_frame in non_nominal_data_frames:
            model = IsolationForest(contamination=0.0002)  # Adjust contamination parameter as needed
            model.fit(nominal_data)
            predicted = model.predict(non_nominal_data_frame)
            anomalies.append(predicted)
        return anomalies


if __name__ == "__main__":
    app = AnomalyDetectionApp()
    app.mainloop()


In [7]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import pandas as pd
from sklearn.ensemble import IsolationForest

class FileSelectionPage(ttk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.initialize()

    def initialize(self):
        label_nominal = ttk.Label(self, text="Select Nominal Excel File:")
        label_nominal.pack(pady=10)

        self.nominal_file_entry = ttk.Entry(self, width=50, state="readonly")
        self.nominal_file_entry.pack(pady=5)

        nominal_file_button = ttk.Button(self, text="Browse", command=self.browse_nominal_file)
        nominal_file_button.pack(pady=5)

        label_non_nominal = ttk.Label(self, text="Select Non-Nominal Excel Files:")
        label_non_nominal.pack(pady=10)

        self.non_nominal_files_entry = ttk.Entry(self, width=50, state="readonly")
        self.non_nominal_files_entry.pack(pady=5)

        non_nominal_files_button = ttk.Button(self, text="Browse", command=self.browse_non_nominal_files)
        non_nominal_files_button.pack(pady=5)

        next_button = ttk.Button(self, text="Next", command=self.go_to_parameter_selection_page)
        next_button.pack(pady=10)

    def browse_nominal_file(self):
        nominal_file_path = filedialog.askopenfilename(filetypes=[("Excel Files", "*.xlsx")])
        self.nominal_file_entry.delete(0, tk.END)
        self.nominal_file_entry.insert(0, nominal_file_path)

    def browse_non_nominal_files(self):
        non_nominal_files_paths = filedialog.askopenfilenames(filetypes=[("Excel Files", "*.xlsx")])
        non_nominal_files_str = ", ".join(non_nominal_files_paths)
        self.non_nominal_files_entry.delete(0, tk.END)
        self.non_nominal_files_entry.insert(0, non_nominal_files_str)

    def go_to_parameter_selection_page(self):
        nominal_file_path = self.nominal_file_entry.get()
        non_nominal_files_paths = self.non_nominal_files_entry.get().split(", ")

        if not nominal_file_path or not non_nominal_files_paths:
            messagebox.showerror("Error", "Please select both nominal and non-nominal files.")
            return

        self.controller.nominal_file_path = nominal_file_path
        self.controller.non_nominal_files_paths = non_nominal_files_paths
        print("Nominal File Path:", self.controller.nominal_file_path)
        print("Non-Nominal Files Paths:", self.controller.non_nominal_files_paths)
        self.controller.show_page("ParameterSelectionPage")


class ParameterSelectionPage(ttk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.initialize()

    def initialize(self):
        parameters_label = ttk.Label(self, text="Select Parameters to Plot (separated by comma):")
        parameters_label.pack(pady=10)

        self.parameters_entry = ttk.Entry(self, width=50)
        self.parameters_entry.pack(pady=5)

        plot_button = ttk.Button(self, text="Plot Selected Parameters", command=self.plot_selected_parameters)
        plot_button.pack(pady=10)

    def plot_selected_parameters(self):
        selected_parameters = [param.strip() for param in self.parameters_entry.get().split(',')]
        if not all(selected_parameters):
            messagebox.showerror("Error", "Please enter parameters to plot.")
            return

        self.controller.selected_parameters = selected_parameters
        self.controller.show_page("PlotPage")


class PlotPage(ttk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.initialize()

    def initialize(self):
        print("Nominal File Path in PlotPage:", self.controller.nominal_file_path)  # Debugging statement
        nominal_data = pd.read_excel(self.controller.nominal_file_path, index_col=0)
        non_nominal_data_frames = [pd.read_excel(file, index_col=0) for file in self.controller.non_nominal_files_paths]

        fig = Figure(figsize=(14, 10))  # Increase the figure size
        ax = fig.add_subplot(111)

        selected_parameters = self.controller.selected_parameters

        for i, non_nominal_data_frame in enumerate(non_nominal_data_frames):
            for parameter in selected_parameters:
                ax.plot(non_nominal_data_frame.index, non_nominal_data_frame[parameter], marker='o', linestyle='-',
                        label=f"File {i + 1}, Parameter {parameter}")

        ax.set_title("Plot of Selected Parameters")
        ax.set_xlabel('Time')  # Set x-axis label to 'Time'
        ax.set_ylabel('Value')
        ax.legend()

        canvas = FigureCanvasTkAgg(fig, master=self)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        toolbar = NavigationToolbar2Tk(canvas, self)
        toolbar.update()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)


class AnomalyDetectionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Anomaly Detection and Root Cause Analysis")

        self.pages = {}
        self.selected_parameters = []
        self.nominal_file_path = None
        self.non_nominal_files_paths = None

        self.initialize_pages()
        self.show_page("FileSelectionPage")

    def initialize_pages(self):
        # File selection page
        file_selection_page = FileSelectionPage(self, self)
        file_selection_page.pack(side="top", fill="both", expand=True)
        self.pages["FileSelectionPage"] = file_selection_page

        # Parameter selection page
        parameter_selection_page = ParameterSelectionPage(self, self)
        parameter_selection_page.pack(side="top", fill="both", expand=True)
        self.pages["ParameterSelectionPage"] = parameter_selection_page

        # Plot page
        plot_page = PlotPage(self, self)
        plot_page.pack(side="top", fill="both", expand=True)
        self.pages["PlotPage"] = plot_page

    def show_page(self, page_name):
        page = self.pages[page_name]
        page.tkraise()


if __name__ == "__main__":
    app = AnomalyDetectionApp()
    app.mainloop()


Nominal File Path in PlotPage: None


ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [9]:
import tkinter as tk
from tkinter import filedialog, ttk
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure


class AnomalyDetectionApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        # Set up main container
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.pages = {}

        # Add pages to the application
        for Page in (FileSelectionPage, ParameterSelectionPage, PlotPage):
            page = Page(container, self)
            self.pages[Page.__name__] = page
            page.grid(row=0, column=0, sticky="nsew")

        # Show the file selection page initially
        self.show_page("FileSelectionPage")

        self.nominal_file_path = None
        self.non_nominal_files_paths = []

    def show_page(self, page_name):
        page = self.pages[page_name]
        page.tkraise()


class FileSelectionPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller

        label = tk.Label(self, text="File Selection Page")
        label.pack(pady=10)

        # Add your file selection logic here

        # Set the nominal_file_path when the nominal file is selected
        self.controller.nominal_file_path = "D:/Data/Book12.xlsx"  # Replace with the actual path

        button = tk.Button(self, text="Next", command=self.goto_parameter_selection)
        button.pack(pady=10)

    def goto_parameter_selection(self):
        self.controller.show_page("ParameterSelectionPage")


class ParameterSelectionPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller

        label = tk.Label(self, text="Parameter Selection Page")
        label.pack(pady=10)

        # Add your parameter selection UI here

        button = tk.Button(self, text="Next", command=self.goto_plot)
        button.pack(pady=10)

    def goto_plot(self):
        self.controller.show_page("PlotPage")


class PlotPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller

        self.initialize()

    def initialize(self):
        print("Nominal File Path in PlotPage:", self.controller.nominal_file_path)  # Debugging statement

        nominal_data = pd.read_excel(self.controller.nominal_file_path, index_col=0)

        # Add your plotting logic here


if __name__ == "__main__":
    app = AnomalyDetectionApp()
    app.mainloop()


Nominal File Path in PlotPage: D:/Data/Book12.xlsx


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DEEPAK PRASAD\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\DEEPAK PRASAD\AppData\Local\Temp\ipykernel_29040\806098989.py", line 85, in plot_selected_parameters
    self.controller.show_page("PlotPage")
  File "C:\Users\DEEPAK PRASAD\AppData\Local\Temp\ipykernel_29040\806098989.py", line 153, in show_page
    page = self.pages[page_name]
KeyError: 'PlotPage'


In [14]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt

class AnomalyDetectionApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.pages = {}

        for F in (FileSelectionPage, PlotPage):
            page_name = F.__name__
            frame = F(parent=container, controller=self)
            self.pages[page_name] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_page("FileSelectionPage")

    def show_page(self, page_name):
        page = self.pages[page_name]
        page.tkraise()

class FileSelectionPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        self.nominal_file_path = None
        self.non_nominal_files_paths = []

        label = tk.Label(self, text="File Selection Page")
        label.pack(pady=10)

        button_nominal = tk.Button(self, text="Select Nominal File", command=self.select_nominal_file)
        button_nominal.pack(pady=10)

        button_non_nominal = tk.Button(self, text="Select Non-Nominal Files", command=self.select_non_nominal_files)
        button_non_nominal.pack(pady=10)

        button_next = tk.Button(self, text="Next", command=self.goto_plot_page)
        button_next.pack(pady=10)

    def select_nominal_file(self):
        self.nominal_file_path = filedialog.askopenfilename(title="Select Nominal File")
        if self.nominal_file_path:
            print("Nominal File Selected:", self.nominal_file_path)

    def select_non_nominal_files(self):
        self.non_nominal_files_paths = filedialog.askopenfilenames(title="Select Non-Nominal Files")
        if self.non_nominal_files_paths:
            print("Non-Nominal Files Selected:", self.non_nominal_files_paths)

    def goto_plot_page(self):
        if self.nominal_file_path and self.non_nominal_files_paths:
            self.controller.show_page("PlotPage")
        else:
            print("Please select both nominal and non-nominal files.")

class PlotPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller

        label = tk.Label(self, text="Plot Page")
        label.pack(pady=10)

        self.selected_param = tk.StringVar(self)
        self.parameters_dropdown = ttk.Combobox(self, textvariable=self.selected_param)
        self.parameters_dropdown.pack(pady=10)

        button_plot = tk.Button(self, text="Plot", command=self.plot_data)
        button_plot.pack(pady=10)

    def plot_data(self):
        selected_param = self.selected_param.get()
        print("Selected Parameter:", selected_param)

        # Add code to plot data here

    def load_data(self):
        all_files = [self.controller.pages["FileSelectionPage"].nominal_file_path] + self.controller.pages["FileSelectionPage"].non_nominal_files_paths
        all_data = pd.concat([pd.read_excel(file) for file in all_files])
        return all_data.columns.tolist()

    def update_dropdown(self):
        params = self.load_data()
        self.parameters_dropdown['values'] = params

if __name__ == "__main__":
    app = AnomalyDetectionApp()
    app.mainloop()


Nominal File Selected: D:/Data/Book12.xlsx
Non-Nominal Files Selected: ('D:/Data/Book13.xlsx',)
